# TODO?

- [G] border refine
- [A] obvious FP
- [ ] probably FP (expert)

- [AMGS] PAPERS PLEASE
- [ ] annotation
- [ ] additional data


- redo grad decent on images , border ref
- sclerotic Gs
- collects stats from raster, redo analysis

# Imports

In [ ]:
from pathlib import Path

import cv2
import rasterio
from tqdm import tqdm
from math import atan2
from shapely import geometry
import numpy as np
from numpy.linalg import eig, inv, svd

import matplotlib.pyplot as plt
%matplotlib inline

# Code

In [ ]:
def __fit_ellipse(x, y):
    x, y = x[:, np.newaxis], y[:, np.newaxis]
    D = np.hstack((x * x, x * y, y * y, x, y, np.ones_like(x)))
    S, C = np.dot(D.T, D), np.zeros([6, 6])
    C[0, 2], C[2, 0], C[1, 1] = 2, 2, -1
    U, s, V = svd(np.dot(inv(S), C))
    a = U[:, 0]
    return a

def ellipse_center(a):
    b, c, d, f, g, a = a[1] / 2, a[2], a[3] / 2, a[4] / 2, a[5], a[0]
    num = b * b - a * c
    x0 = (c * d - b * f) / num
    y0 = (a * f - b * d) / num
    return np.array([x0, y0])

def ellipse_axis_length(a):
    b, c, d, f, g, a = a[1] / 2, a[2], a[3] / 2, a[4] / 2, a[5], a[0]
    up = 2 * (a * f * f + c * d * d + g * b * b - 2 * b * d * f - a * c * g)
    down1 = (b * b - a * c) * (
        (c - a) * np.sqrt(1 + 4 * b * b / ((a - c) * (a - c))) - (c + a)
    )
    down2 = (b * b - a * c) * (
        (a - c) * np.sqrt(1 + 4 * b * b / ((a - c) * (a - c))) - (c + a)
    )
    res1 = np.sqrt(up / down1)
    res2 = np.sqrt(up / down2)
    return np.array([res1, res2])

def ellipse_angle_of_rotation(a):
    b, c, d, f, g, a = a[1] / 2, a[2], a[3] / 2, a[4] / 2, a[5], a[0]
    return atan2(2 * b, (a - c)) / 2

def fit_ellipse(x, y):
    """@brief fit an ellipse to supplied data points: the 5 params
        returned are:

        M - major axis length
        m - minor axis length
        cx - ellipse centre (x coord.)
        cy - ellipse centre (y coord.)
        phi - rotation angle of ellipse bounding box

    @param x first coordinate of points to fit (array)
    @param y second coord. of points to fit (array)
    """
    a = __fit_ellipse(x, y)
    centre = ellipse_center(a)
    phi = ellipse_angle_of_rotation(a)
    M, m = ellipse_axis_length(a)
    # assert that the major axix M > minor axis m
    if m > M:
        M, m = m, M
    # ensure the angle is betwen 0 and 2*pi
    phi -= 2 * np.pi * int(phi / (2 * np.pi))
    return [M, m, centre[0], centre[1], phi]

def _generate_ell(a, b, center0, center1, phi):
    arc = 2.0
    R = np.arange(0, arc * np.pi, 0.1)
    center, axes = (center0, center1), (a, b)

    a, b = axes
    xx = center[0] + a * np.cos(R) * np.cos(phi) - b * np.sin(R) * np.sin(phi)
    yy = center[1] + a * np.cos(R) * np.sin(phi) + b * np.sin(R) * np.cos(phi)
    return xx,yy

def plot_ellipse(x,y,xx,yy):
    plt.figure(0)
    plt.plot(x, y, color='blue', label='points')
    plt.plot(xx, yy, '+', color='red', label='fitted ellipse', linewidth=2.0)
    plt.legend()
    plt.axes().set_aspect('equal', 'datalim')
    #plt.savefig('plot' + str(sample_i) + '.png')
    #plt.clf()
    
def create_clone(poly):
    '''
    FITS ELLIPS INTO POLY
    '''
    x,y = gpoly(poly)
    xm,ym = x.mean(), y.mean() 
    x-=xm
    y-=ym
        
    params = fit_ellipse(x,y)
    xx,yy = _generate_ell(*params)
    xx+=xm
    yy+=ym
    cds = np.concatenate([xx[None,:],yy[None,:]], axis=0).T
    new_poly = geometry.Polygon(cds)
    return new_poly, params

def iou(p1,p2):
    if not p1 or not p2:
        print(f'ERR POLY: {p1}, {p2}')
    try:
        i = p1.intersection(p2)
        u = p1.union(p2)
    except Exception as e:
        print(e)
        return 0
        
    return i.area/u.area

In [ ]:
def read_polys(data):
    polys = []
    for d in data:
        cd = d['geometry']['coordinates'][0]
        try:
            poly = geometry.Polygon(cd)
        except Exception as e:
            print(cd)
            
        polys.append(poly)
    return polys

def poly_stats(polys):
    stats = {}
    a = []
    for p in polys:
        a.append(p.area)
    
    stats['MEAN_AREA'] = np.array(a).mean().round()
    
    return stats

def gpoly(poly):
    x,y = poly.exterior.xy
    x, y = np.array(x), np.array(y)
    return x, y
    
def plot_polys_2d(polys, ax):
    for p in polys:
        x,y = gpoly(p)
        x-=x.mean()
        y-=y.mean()
        ax.plot(x,y, alpha=.5)

def IA(axs):
    for i in range(len(axs)):
        for j in range(len(axs[0])):
            yield axs[i][j]

# ACTION!

In [ ]:
import json

In [ ]:
jss = Path('../input/hm/train').glob('*.json')
jss = list(jss)
all_polys = {}
for js in jss:
    if '-' in js.name or '_' in js.name:
        # anatomical structures
        continue
        
    with open(js, 'r') as f:
        polys = read_polys(json.load(f))
        print(f'FILE: {js} : {len(polys)} polygons total')
        all_polys[js] = polys

In [ ]:
n,m = 4,4
f, axs = plt.subplots(n,m, sharex=True, sharey=True, figsize=(16,16))
ia = IA(axs)

for f,ps in all_polys.items():
    plot_polys_2d(ps, next(ia))

In [ ]:
import rasterio as rio
import scipy

In [ ]:
def get_poly_raster(img_name, p):
    x1,y1,x2,y2 = p.bounds
    ds = rio.open(str(img_name))
    rast = ds.read(window=((y1,y2),(x1,x2)))
    return rast.transpose(1,2,0)

def get_poly_mask(p):
    p = (p - p.centroid)
    x,y = gpoly(p)
    x-=x.mean()
    y-=y.mean()
    x+=abs(x.min())
    y+=abs(y.min())
    p = geometry.Polygon(np.array([x,y]).T)
    x1,y1,x2,y2 = p.bounds
    buf = np.zeros((int(y2-y1),int(x2-x1), 3)).astype(np.uint8)
    
    pts = np.array(p.exterior.xy).T.astype(np.int32)
    cv2.fillPoly(buf, np.expand_dims(pts,0), (255,255,255));
    
    return buf

In [ ]:
def gen_stats(img_name, polys):
    for p in tqdm(polys):
        mask = get_poly_mask(p)
        rast = get_poly_raster(img_name, p)
        gpixels = rast[mask>0]
        area, color = p.area**.5, gpixels.mean()
        yield area, color

def get_poly_scores(img_name, polys):
    all_stats = np.array(list(gen_stats(img_name, polys)))
    areas, colors = all_stats[:,0], all_stats[:,1]
    
    scores = []
    for a, c in all_stats:
        p1 = abs(scipy.stats.percentileofscore(areas, a) / 100 - .5)
        p2 = abs(scipy.stats.percentileofscore(colors, c) / 100 - .5)
        p1 *= 2
        p2 *= 2
        score = p1 * p2
        scores.append(score)
    return np.array(scores)

In [ ]:
def read_polys(data):
    polys = []
    for d in data:
        cd = d['geometry']['coordinates'][0]
        try:
            poly = geometry.Polygon(cd)
        except Exception as e:
            print(cd)
            
        polys.append(poly)
    return polys


def get_polys(img_name):
    js = img_name.with_suffix('.json')
        
    with open(js, 'r') as f:
        polys = read_polys(json.load(f))
    
    return polys

In [ ]:
imgs = list(Path('../input/hm/train/').glob('*.tiff'))
img_name = imgs[2]

In [ ]:
polys = get_polys(img_name)

In [ ]:
scores = get_poly_scores(img_name, polys)

In [ ]:
class WeightedDataset:
    def __init__(self, dataset, scores, replacement=True):
        assert len(dataset) != len(scores), (len(dataset), len(scores))
        scores = ( 1+ scores) ** 2
        self.scores = scores / scores.sum()
        self.dataset = dataset
        self.replacement = replacement
        self.idxs = list(range(len(self.dataset)))
        #self.sampler = WeightedRandomSampler(self.scores, len(self.scores), replacement=replacement)
    
    def __getitem__(self, _):
        num_samples = 1
        idx = np.random.choice(self.idxs, num_samples, self.replacement, self.scores)
        #idxs = list(self.sampler)
        return self.dataset(idx)
        
    def __len__(self): return len(self.dataset)

In [ ]:
plt.hist(scores, bins=20);

In [ ]:
plt.hist(res[:,0], bins=20);

In [ ]:
plt.hist(res[:,1], bins=20);

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

k = ks[0]
polys = all_polys[k]
scores = p1s * p2s

@interact(idx=(0, len(polys)-1),continuous_update=False)
def view(idx):
    p = polys[idx]
    score = scores[idx]
    mask = get_poly_mask(p)
    rast = get_poly_raster(k.with_suffix('.tiff'), p)
    gpixels = rast[mask>0]
    plt.imshow(rast)
    plt.imshow(mask, alpha=.2)
    
    print(gpixels.mean(), gpixels.std(), score)

In [ ]:
np.array(mask.shape).prod()//3

### Advanced stats

In [ ]:
l = []
for f,ps in all_polys.items():
    #print(f, poly_stats(ps))
    a = poly_stats(ps)['MEAN_AREA']
    r = a**.5
    print(f,r)
    l.append((f,r))

In [ ]:
sl = sorted(l, key=lambda x: x[1])

In [ ]:
scales = {'b9a3865fc': .65,
'e79de561c': .5,
'8242609fa': .65,
'cb2d976f4': .65,
'b2dc8411c': .65,
'afa5e8098': .5,
'0486052bb': .65,
'aaa6a05cc': .65,
'2f6ecfcdf': .65}

domains = {'4ef6695ce': 1,
 'b9a3865fc': 0,
 'e79de561c': 1,
 '8242609fa': 0,
 'cb2d976f4': 0,
 '26dc41664': 1,
 'b2dc8411c': 0,
 'afa5e8098': 1,
 '0486052bb': 0,
 '1e2425f28': 1,
 'c68fe75ea': 1,
 'aaa6a05cc': 0,
 '54f2eec69': 1,
 '095bf7a1f': 1,
 '2f6ecfcdf': 0}



In [ ]:
dd = .65/.5
a,b = [],[]
for s in sl:
    name = s[0].stem
    sc = scales.get(name, 0)
    d = dd if not domains[name] else 1
    if domains[name]:
        a.append(int(s[1]))
    else:
        b.append(int(s[1]))
        
    print(s[0].name,  domains[name], int(s[1]), int(s[1] * d), sc)

In [ ]:
np.array(a).mean(), np.array(b).mean()

# CLONE WARS

In [ ]:
first_img_polys = list(all_polys.values())[0]
poly = first_img_polys[1]
x,y = gpoly(poly)
params = fit_ellipse(x,y)
xx,yy = _generate_ell(*params)

plot_ellipse(x,y,xx,yy)

In [ ]:
all_ious = {}
for f,ps in all_polys.items():
    ious = []
    for p in ps:
        pp, _ = create_clone(p)
        ious.append(iou(p,pp))
        #break
    ious = np.array(ious)
    all_ious[f] = ious
    print(f'FILE {f} IOU {ious.mean()}')

In [ ]:
n,m = 2,4
f, axs = plt.subplots(n,m, sharex=True, sharey=True, figsize=(16,8))
ia = IA(axs)

for f,ious in all_ious.items():
    ax = next(ia)
    l,h = np.percentile(ious, 1), np.percentile(ious, 99)
    ious = ious[(ious > l) & (ious < h)]
    ax.hist(ious, bins=50)
    ax.set_label(f)

In [ ]:
ell = {}
for f,ps in all_polys.items():
    elons = []
    for p in ps:
        _, params = create_clone(p)
        elons.append(params[0]/params[1])
        
    elons = np.array(elons)
    ell[f] = elons
    print(f'FILE {f} ELONGATION {elons.mean()}')

# DUMP

In [ ]:
def dump_ann_d(path, d):
    for fn, ann in d.items():
        with open(str(path/(fn.with_suffix('').name + '_ell.json')), 'w') as f:
            json.dump(ann,f)
        

In [ ]:



new_anns = {}
for f,ps in all_polys.items():
    ann = []
    for p in ps:
        pp, _ = create_clone(p)
        try:
            x,y = gpoly(pp)
        except Exception as e:
            print(e)
            continue
        
        cds = np.concatenate([x[None,:],y[None,:]], axis=0).T.tolist()
        template = {'type': 'Feature',
 'id': 'PathDetectionObject',
 'geometry': {'type': 'Polygon',
  'coordinates': []},
 'properties': {'classification': {'name': 'glomerulus', 'colorRGB': -3140401},
  'isLocked': False,
  'measurements': [{'name': 'detection_score', 'value': 0.123}]}}
        t = template.copy()
        t['geometry']['coordinates'] = [cds]
        ann.append(t)
        #break
    new_anns[f] = ann

In [ ]:
new_anns.keys()

In [ ]:
pa = Path('/home/sokolov/work/webinf/data//kidney/train/')
dump_ann_d(pa, new_anns)

In [ ]:
jss = Path('/home/sokolov/work/webinf/data//kidney/train/').glob('*_ell.json')
jss = list(jss)
tt = {}
for js in jss:
    if '-' in js.name:
        # anatomical structures
        continue
        
    with open(js, 'r') as f:
        polys = read_polys(json.load(f))
        print(f'FILE: {js} : {len(polys)} polygons total')
        tt[js] = polys

In [ ]:
list(tt.values())[0][3].exterior.xy

# END

In [ ]:
def fivePointsToConic(points, f=1.0):
    """Solve for the coefficients of a conic given five points in Numpy array

    `points` should have at least five rows.

    `f` is the constant that you can specify. With the returned solution,
    `(a, b, c, d, e, f)`, the full conic is specified as:

    $a x^2 + b x y + c y^2 + d x + e y = -f$

    If `points` has exactly five rows, the equation will be exact. If `points`
    has *more* than five rows, the solution will be a least-squares one that
    fits the data the best.
    """
    from numpy.linalg import lstsq

    x = points[:, 0]
    y = points[:, 1]
    if max(x.shape) < 5:
        raise ValueError('Need >= 5 points to solve for conic section')

    A = np.vstack([x**2, x * y, y**2, x, y]).T
    fullSolution = lstsq(A, f * np.ones(x.size),rcond=True)
    (a, b, c, d, e) = fullSolution[0]
    return (a, b, c, d, e)


In [ ]:
points = np.array([[-2, 4.], [-1., 1], [0., 0], [1., 1], [2., 4]])
f=-1
(a, b, c, d, e) = fivePointsToConic(points,f)

In [ ]:
x = np.linspace(-5, 5, 100)
y = np.linspace(0, 5, 100)
x, y = np.meshgrid(x, y)

In [ ]:
def axes():
    plt.axhline(0, alpha=.1)
    plt.axvline(0, alpha=.1)

In [ ]:
c=b=e=0

In [ ]:
a,c,f = 3,3, -9

In [ ]:
r = 1
sx = np.linspace(-1, 1, 30)
sy = (r**2-(sx**2))**.5
sx = sx[None,].T
sy = sy[None,].T

#spoints = np.array([(b,a) for a,b in zip(sx,sy)])
#spoints

In [ ]:
A = np.hstack([sx**2, sx * sy, sy**2, sx, sy])
b = np.ones_like(sx)
r = np.linalg.lstsq(A, b, rcond=True)[0].squeeze()
r

In [ ]:
f=9
(a, b, c, d, e) = fivePointsToConic(spoints,f)
(a, b, c, d, e)

In [ ]:
axes()
plt.contour(x, y, (a*x**2 + b*x*y + c*y**2 +d*x +e*y + f), [0], colors='k')
plt.plot(spoints[:,0], spoints[:,1], marker='o')
plt.show()

In [ ]:
plt.plot(points[:,0], points[:,1], marker='o')